In [1]:
"""
Extract gridMET variables 
Google Earth Engine (GEE) Python API
Author: maxwell.cook@colorado.edu
"""

import os, sys
import ee
import geemap
import time

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

print("Success !")

Success !


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [2]:
# Load fire perimeters
fires = ee.FeatureCollection('projects/jfsp-aspen/assets/nifc_aspenfires_2018to2023')
print(f"Number of aspen fires: {fires.size().getInfo()}")
print(fires.first().propertyNames().getInfo())

Number of aspen fires: 50
['Fire_Year', 'Fire_ID', 'Ig_Date', 'Last_Date', 'Fire_Name', 'system:index']


In [3]:
# Load the gridmet image collection

In [4]:
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
print(f"gridMET bands available for analysis:\n\n{gridmet.first().bandNames().getInfo()}")

gridMET bands available for analysis:

['pr', 'rmax', 'rmin', 'sph', 'srad', 'th', 'tmmn', 'tmmx', 'vs', 'erc', 'eto', 'bi', 'fm100', 'fm1000', 'etr', 'vpd']


In [5]:
# select our variables of interest:
# 1. Vapor Pressure Deficit and 2. Energy Release Component
gridmet = gridmet.select(['vpd','erc'])

In [6]:
# calculate the 15 year average VP, ERC
mean15 = gridmet.filter(ee.Filter.calendarRange(2008,2024,'year')).mean() 
gridmet.first().bandNames().getInfo()

['vpd', 'erc']

In [7]:
# Load the gridded FRP data for aspen fires in the Southern Rockies

In [8]:
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/viirs_snpp_jpss1_afd_aspenfires_gridstats')
print(f"Number of gridcells: {grid.size().getInfo()}")
print(grid.first().propertyNames().getInfo())

Number of gridcells: 49343
['Fire_Year', 'grid_index', 'Fire_ID', 'max_date', 'afd_count', 'Ig_Date', 'first_date', 'last_date', 'Last_Dat_1', 'system:index']


In [9]:
# get a list of unique active fire days
fire_days = grid.aggregate_array('max_date').getInfo()
fire_days = set(fire_days)
print(f"Number of 'fire days': {len(fire_days)}")

Number of 'fire days': 360


In [10]:
# map over the list of unique dates
def gridmet_summary(fireID):
    """ calculate the daily gridmet for active fire detections """
    # grab the fire id
    # filter obs. to the day of burning
    # get the unique dates of max FRP
    fire_grid = grid.filter(ee.Filter.eq('Fire_ID', fireID))
    fire_days = ee.List(fire_grid.aggregate_array('max_date')).distinct()
    
    def get_daily(day):
        """ calculate the daily gridmet summary"""
        # filter to this day's detections
        # retrieve the date information for filtering
        # get bounds for all grids in the same day
        grids_day = fire_grid.filter(ee.Filter.eq('max_date', day))
        bounds = grids_day.geometry().bounds() # gather the bounds for this days detections
        max_frp_day = ee.Date(day) # the day of maximum FRP
        prev_day = max_frp_day.advance(-1,'days') # previous day for calculating the mean
        
        # calculate the gridmet mean
        # calculation the deviation from the 15-year average
        gridmet_ = gridmet.filterDate(prev_day, max_frp_day).mean()
        attrs = ee.List(gridmet_.bandNames()) # grab the band names for renaming
        attrs = attrs.map(lambda atr: ee.String(atr).cat('_dv'))
        anomaly = gridmet_.subtract(mean15).rename(attrs)
        combined = gridmet_.addBands(anomaly) # combine the bands
        
        # run the reduction
        stats = combined.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=bounds,
            scale=4000
        )

        return ee.Feature(None, stats.set("max_date", day))

    results = fire_days.map(get_daily)
    return ee.FeatureCollection(results).set('Fire_ID', fireID)

print("Function ready !")

Function ready !


In [12]:
# get a list of fire names
fire_ids = grid.aggregate_array('Fire_ID').distinct()
print(f"Processing {fire_ids.size().getInfo()} fires.")

# run the gridmet summary for fires
def map_fires(fireID):
    return gridmet_summary(fireID)

grid_stats = ee.FeatureCollection(
    fire_ids.map(map_fires).flatten()
)

print("Number of features:", grid_stats.size().getInfo())
print("First feature properties:", grid_stats.toDictionary().getInfo())
print("Submitted !")

Processing 54 fires.
Number of features: 54
First feature properties: {}
Submitted !


In [17]:
# Export the table. 
export_task = ee.batch.Export.table.toDrive(
    collection=grid_stats,
    description='viirs_snpp_jpss1_afd_aspenfires_gridstats_gridmet',
    fileNamePrefix='viirs_snpp_jpss1_afd_aspenfires_gridstats_gridmet',
    fileFormat='CSV', 
    folder='GRIDMET'
)

export_task.start() # Start the export task
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 240) 

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting fo